<a href="https://colab.research.google.com/github/Swathi1309/ED18B034_ME18B133_Assignment1/blob/main/Assignment1_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wandb
!wandb login

     |████████████████████████████████| 2.0MB 5.7MB/s 
     |████████████████████████████████| 133kB 31.2MB/s 
     |████████████████████████████████| 102kB 9.1MB/s 
     |████████████████████████████████| 163kB 27.2MB/s 
     |████████████████████████████████| 71kB 7.1MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=498c4566abce21d04ed9a097346ba76b61234af03a68e97e8af71d7e786668f7
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=41490cb54757523c5fe92431889a0ec5978f0db2ac78f18c24264473d7e80551
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
Successfully built subprocess32 pathtools
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: App

In [1]:
import keras
from keras.datasets import fashion_mnist
from PIL import Image
# import wandb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import math
import sklearn

# wandb.init(project="assignment-1", entity="swathi")
# classes = ['T-shirt/top','Trouser','Pullover','Dress','Coat','Sandal','Shirt','Sneaker','Bag','Ankle boot']

In [ ]:
def load_images():
  (x_train,y_train),(x_test,y_test) = fashion_mnist.load_data()
  X_train_img = x_train[:54000]
  X_validation_img = x_train[54000:]
  X_test_img = x_test
  return X_train_img, X_validation_img, X_test_img

In [2]:
def load_data():
  (x_train,y_train),(x_test,y_test) = fashion_mnist.load_data()
  X_train = np.transpose((np.array(x_train[:54000],dtype=int).reshape([54000,784]))/255)
  Y_train = np.transpose((np.array(y_train[:54000],dtype=int).reshape([54000,1])))
  X_validation = np.transpose((np.array(x_train[54000:],dtype=int).reshape([6000,784]))/255)
  Y_validation = np.transpose(np.array(y_train[54000:],dtype=int).reshape([6000,1]))
  X_test = np.transpose((np.array(x_test,dtype=int).reshape([10000,784]))/255)
  Y_test = np.transpose(np.array(y_test,dtype=int).reshape([10000,1]))
  return X_train, Y_train, X_validation, Y_validation, X_test, Y_test

In [ ]:
def log_images():
  images = []
  labels = []
  for i in range(len(X_train_img)):
    if len(labels)==len(classes):
      break
    j = random.randint(0,53999)
    if classes[Y_train[j,0]] not in labels:
      images.append(X_train_img[j])
      labels.append(classes[Y_train[j,0]])
  wandb.log({"Examples": [wandb.Image(img, caption=caption) for img,caption in zip(images,labels)]})

Loading data

In [3]:
X_train, Y_train, X_validation, Y_validation, X_test, Y_test = load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [ ]:
log_images()

IndexError: ignored

Defining basic parameters

In [4]:
N= [0,64,32,16,10]  #Insert the number of neurons in each layer
N[0]=X_train.shape[0]
l = len(N) - 1

Weights initialising

In [5]:
def initialize_weights(N):
  np.random.seed(0)
  W = []
  b = []
  W.append(0)
  b.append(0)
  for i in range(1,l+1):
    W.append(np.random.randn(N[i],N[i-1]))
    b.append(np.zeros((N[i],1)))

  return W,b
  

In [6]:
def one_hot(y):
  one_hot_y=np.zeros((10,X_train.shape[1]))
  for i in range(X_train.shape[1]):
    one_hot_y[y[0,i],i] = 1
  
  return one_hot_y

In [7]:
print (one_hot(Y_train)[:,7])

[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]


Softmax

In [6]:
def softmax(a):
  return np.exp(a)/sum(np.exp(a))

Sigmoid

In [7]:
def sigmoid(a):

  return 1/(1+np.exp(-a))


Sigmoid derivative

In [8]:
def dSigmoid(a):

  return sigmoid(a)*(1-sigmoid(a))

Cross Entropy

In [14]:
def calculate_loss(y_hat,Y):
  loss = 0
  for i in range(Y.shape[1]):
    loss -= np.log(y_hat[Y[0,i],i])
  print (loss)

Feed forward network

In [9]:
def feedforward(X_input, W, b, N, l):
  a=[]
  a.append(0)
  h=[]
  h.append(X_input)
  for i in range(1,l):
    # print(i)
    #print (W[i].shape)
    #print (h[i-1].shape)
    #print (b[i].shape)
    a_new=b[i]+np.matmul(W[i],h[i-1])
    h_new=sigmoid(a_new)
    a.append(a_new)
    h.append(h_new)

  a_new=b[l]+np.matmul(W[l],h[l-1])
  y_hat=softmax(a_new)
  a.append(a_new)

  return a,h,y_hat

Backpropogation

In [10]:
def backrop(y,y_hat,W,a,h):
  grad_W=[]
  grad_b=[]
  grad_a=[]
  grad_h=[]
  
  da = y_hat - one_hot(y)
  grad_a.append(da)

  for i in reversed(range(1,l+1)):
    dw = np.matmul(da,np.transpose(h[i-1]))
    db = np.sum(da, axis=1, keepdims=True)
    dh_prev = np.matmul(np.transpose(W[i]),da)
    da_prev = dh_prev*dSigmoid(a[i-1])
    
    grad_W.append(dw)
    grad_b.append(db)
    grad_a.append(da_prev)
    grad_h.append(dh_prev)
    da=da_prev

  grad_W.append(0)
  grad_b.append(0)
  grad_W.reverse()
  grad_b.reverse()
  grad_a.reverse()
  grad_h.reverse()
  

  return grad_W,grad_b,grad_a,grad_h


Gradient Descent

In [11]:
def grad_descent(W,b,grad_W,grad_b,learn_rate=0.001):
  W_new=[]
  W_new.append(0)
  b_new=[]
  b_new.append(0)
  m=X_train.shape[1]
  for i in range(1,l+1):
    #print(W[i].shape,grad_W[i].shape,b[i].shape,grad_b[i].shape)
    W_new.append(W[i] - (learn_rate*grad_W[i])/m)
    b_new.append(b[i] - (learn_rate*grad_b[i])/m)
  return W_new,b_new


In [ ]:
def gradient_descent(epochs, learn_rate):
  W, b = initialize_weights()
  W_new = W
  b_new = b
  for i in range(epochs):
    a, h, y_hat = feed_forward(W,b)
    calculate_loss(y_hat, Y)
    grad_W, grad_b, grad_h, grad_a = backrop(Y, y_hat, W, a, h)
    for j in range(1,L+1):
      W_new[j] = W[j] - grad_W[j]
      b_new[j] = b[j] - grad_b[j]
    W = W_new
    b = b_new
    #print (b_new[1].shape)
  a, h, y_hat = feed_forward(W,b)
  calculate_loss(y_hat, Y)

Training

In [12]:
def training(N,l,epochs,batch_size,learn_rate,X_input,y):
  W , b = initialize_weights(N)
  n=(np.floor(X_input.shape[1]/batch_size)+1).astype(int)
  for j in range(epochs):
    for j in range(n):
      X=X_input[:,[j*batch_size,(j+1)*batch_size]]
      a , h , y_hat = feedforward(X_input, W, b, N, l) 
      grad_W , grad_b = backrop(y,y_hat,W,a,h)[0:2]
      W , b = grad_descent(W,b,grad_W,grad_b,learn_rate)
    
    calculate_loss(y_hat,y)
  
  return W,b


In [13]:
def predict(W,b,X_test,N,l):
  y_hat = feedforward(X_test, W, b, N, l)[2]
  y_class = np.argmax(y_hat,axis=0).reshape([1,X_test.shape[1]])

  return y_class

In [33]:
W_new,b_new= training(N,l,10,64,X_train,Y_train)

KeyboardInterrupt: ignored

In [ ]:
y_hat=feedforward(X_train, W_new, b_new, N, l)[2]
print(y_hat[:,0:4])

[[7.95438641e-04 5.06923535e-04 2.72384172e-04 2.89546831e-04]
 [5.54728207e-03 8.37974101e-03 9.19908637e-03 9.82112961e-03]
 [5.96238437e-04 3.57710876e-04 6.20491689e-04 5.54516655e-04]
 [1.70443892e-02 4.46402932e-03 6.63305384e-03 4.23114800e-03]
 [1.31518876e-03 1.06975692e-02 1.37143730e-03 2.31333945e-03]
 [2.00000523e-03 1.13297207e-03 6.98664703e-03 4.32605725e-03]
 [6.23692176e-01 6.41550739e-01 1.90095725e-01 1.34658239e-01]
 [2.94504801e-01 2.32681272e-01 6.16109611e-01 6.24439117e-01]
 [3.32833417e-03 4.83720628e-03 6.32890351e-03 5.45661290e-03]
 [5.11761470e-02 9.53918369e-02 1.62382660e-01 2.13910293e-01]]


In [ ]:
Y_predict = predict(W_new,b_new,X_train,N,l)
print (Y_predict[0,0:4])
print (Y_test[0,0:4])
#sklearn.metrics.accuracy_score(Y_test, Y_predict)

[6 6 7 7]
[9 2 1 1]


In [ ]:
sweep_config = {
    'method': 'random'
    }

In [ ]:
parameters_dict = {
    'epochs': {
        'values': [5, 10]
      },
    'l': {
        'values': [3, 4, 5]
      },
    'size of each layer': {
        'values': [32, 64, 128]
      },
    'weight decay': {
        'values': [0, 0.0005, 0.5]
      },
    'learn_rate': {
        'values': [1e-3,1e-4]
      },
    'epochs': {
        'values': [5,10]
      },
    'batch_size': {
        'values': [16,32,64]
      },
    'weights_initialization': {
        'values': ['random', 'Xavier']
      },
    'optimizer': {
        'values': ['sgd', 'mom', 'nest', 'rms', 'adam', 'nadam']
      },
    'act_func': {
          'values': ['sigmoid', 'tanh', 'ReLU']
      }
    }

sweep_config['parameters'] = parameters_dict

In [ ]:
import pprint

pprint.pprint(sweep_config)

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="pytorch-sweeps-demo")

In [ ]:
wandb.agent(sweep_id, train, count=5)

In [ ]:
def training_sweep(N,l,X_input,y,config=None):
  
  # Initialize a new wandb run
    with wandb.init(config=config):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config

        training(N,l,config.epochs,config.batch_size,config.learn_rate,X_input,y)
            wandb.log({"loss": avg_loss, "epoch": epoch})